# What Causes More Scientific Discoveries in Short Time

## Data Clean

- **Creating Author:** Yanheng Liu
- **Latest Modification:** 13-04-2025  
- **Modification Author:** Yanheng Liu  
- **E-mail:** [yanheng.liu@etu.sorbonne-universite.fr](mailto:yanheng.liu@etu.sorbonne-universite.fr)  
- **Version:** 1.2  

---

This is a data clean provided for the project in *DALAS* course.


## Data Cleaning and Deduplication Workflow

### Step 0: Check library installation

### Step 1: Define Common Functions
Includes text cleaning, normalization, and fuzzy matching functions.

### Step 2: Load and Merge Data
Reads three data files and merges them into a single DataFrame.

### Step 3: Clean Punctuation
Removes unnecessary punctuation from all columns (e.g., quotes), but keeps semicolons.

### Step 4: Preview Country Names
Displays original country names to check for inconsistencies manually.

### Step 5: Standardize Country Names
Uses a predefined mapping dictionary to unify country name variations.

### Step 6: Preview Invention Categories
Shows all original invention categories.

### Step 7: Generalize Categories
Maps specific invention categories to broader scientific fields, with detailed explanation.

### Step 8: Normalize Key Text Fields
Cleans and normalizes 'Name of Invention' and 'Name of Inventor' by lowering case and removing stopwords.

### Step 9: Deduplicate Using Fuzzy Matching
Groups similar invention names using fuzzy matching and merges records by combining field values.

### Step 10: Output Final Dataset
Displays the final cleaned dataset and saves it to a new CSV file.


Check package whether are installed in the environment.

In [58]:
import pkg_resources
import subprocess

# Read package list from requirements.txt
with open("../../requirements.txt", "r") as file:
    packages = [line.strip() for line in file if line.strip() and not line.startswith("#")]

# Get the list of currently installed packages
installed_packages = {pkg.key for pkg in pkg_resources.working_set}

# Check and install missing packages
for package in packages:
    pkg_name = package.split("==")[0].lower() if "==" in package else package.lower()
    if pkg_name not in installed_packages:
        print(f"Installing missing package: {package}")
        try:
            subprocess.check_call(["pip", "install", package])
        except subprocess.CalledProcessError as e:
            print(f"Failed to install {package}. Error: {e}")
    else:
        print(f"Already installed: {package}")


Already installed: requests
Already installed: beautifulsoup4
Already installed: pandas
Already installed: tabulate
Already installed: pdfplumber
Already installed: lxml
Already installed: pandas
Already installed: fuzzywuzzy


### Public Functions
The cell is used to import pandas, re, string and fuzzywuzzy related functions, and defines common text preprocessing, punctuation cleaning, country normalisation, invention category classification, text normalisation and fuzzy matching and de-emphasis and other basic functions.

In [59]:
import pandas as pd
import re
import string
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Define a translation table to remove punctuation except semicolon
punctuations = string.punctuation.replace(";", "")  # keep semicolon
trans_table = str.maketrans('', '', punctuations)

def remove_punctuation(text):
    """Remove punctuation from text except semicolon."""
    if isinstance(text, str):
        # Remove punctuation using the translation table
        return text.translate(trans_table).strip()
    else:
        return text

def normalize_text(text, stopwords=None):
    """Normalize text: lower-case, remove punctuation and stopwords, and extra spaces."""
    if not isinstance(text, str):
        return text
    # lower-case
    text = text.lower()
    # remove punctuation (except semicolon)
    text = text.translate(trans_table)
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    if stopwords:
        # remove stopwords provided as a set
        tokens = text.split()
        tokens = [t for t in tokens if t not in stopwords]
        text = ' '.join(tokens)
    return text

def standardize_country(country_str, mapping):
    """
    Standardize country names.
    Split multiple countries by semicolon, map each to official name, then return unique sorted values.
    """
    if not isinstance(country_str, str):
        return country_str
    # Split by semicolon and possibly comma if there is complex cases
    parts = re.split(r'[;，,]', country_str)
    standardized = []
    for part in parts:
        # Remove leading and trailing spaces and convert to lower case
        part_clean = part.strip().lower()
        if part_clean in mapping:
            standardized.append(mapping[part_clean])
        else:
            # If not in the mapping, capitalise the first letter.
            standardized.append(part_clean.title())
    # De-weighting and sorting before joining with a semicolon
    standardized = sorted(set(standardized))
    return "; ".join(standardized)

def generalize_category(cat):
    """
    Map detailed invention categories into broad categories.
    Matching is case‐insensitive and covers all specified categories.
    """
    if not isinstance(cat, str):
        return 'Other'
    s = cat.strip().lower()
    mapping_rules = [
        # Physical Sciences
        (r'\b('
         r'physics|astronomy|astrophysics|cosmology|geophysics|'
         r'nuclear(?: science| physics| weapons| fusion)?|radiometric|'
         r'chaos theory|optics|thermodynamics|acoustics|paleontology|'
         r'measurement instrument|scientific imaging|scientific theory|'
         r'astronomical theory|scientific model|electricity'
         r')\b',
         'Physical Sciences'),

        # Chemistry & Materials
        (r'\b('
         r'chemistry|biochemistry|electrochemistry|chemical engineering|'
         r'materials science|materials technology|metallurgical engineering|'
         r'metallurgical process|metallurgy|explosives|explosive technology|'
         r'paper technology|battery technology|polymer material|plastic material|'
         r'flooring material|chemical discovery|chemical process|ammunition|'
         r'chemical element|nanotechnology|textile fiber'
         r')\b',
         'Chemistry & Materials'),

        # Life Sciences & Medicine
        (r'\b('
         r'biology|molecular biology|microbiology|genetics|genomics|'
         r'evolutionary biology|physiology|neuroscience|neurology|virology|'
         r'immunology|medicine|medical imaging|medical instrument|'
         r'medical anaesthetic|medical equipment|pharmaceuticals|biotechnology'
         r')\b',
         'Life Sciences & Medicine'),

        # Earth & Environmental Sciences
        (r'\b('
         r'geology|climatology|climate science|environmental science|'
         r'oceanography|sanitation technology'
         r')\b',
         'Earth & Environmental Sciences'),

        # Agricultural Sciences & Technology
        (r'\b('
         r'agriculture|agricultural machinery|agricultural engineering|'
         r'agricultural implement|agricultural technology|agricultural chemistry|'
         r'agricultural fencing|food technology|food safety process'
         r')\b',
         'Agricultural Sciences & Technology'),

        # Engineering & Technology
        (r'\b('
         r'mechanical engineering|electrical engineering|electronic engineering|'
         r'civil engineering|construction engineering|structural engineering|'
         r'aerospace engineering|aerospace|mining engineering|industrial engineering|'
         r'hydraulic engineering|aviation(?: technology)?|aeronautical vehicle|'
         r'aeronautical device|rocketry|marine technology|marine navigation|'
         r'marine safety equipment|automotive technology|transportation(?: engineering)?|'
         r'transportation safety device|transport system|railway carriage|'
         r'vehicle innovation|vehicle|construction materials|energy storage device|'
         r'3d printing|diving technology|electric motor|household appliance|'
         r'industrial design|lighting device|military technology|mining technology|'
         r'space technology|technical drawing|timekeeping mechanism|timekeeping technology|'
         r'refrigeration technology|domestic technology|home appliances?|consumer electronics|'
         r'printing technology|printing machinery|industrial machinery|industrial process|'
         r'machine tools?|textile machinery|garment technology|clothing|welding technology|'
         r'lock technology|firearms? technology|weapon technology|sports equipment|turbine|'
         r'engine|logistics|vertical transportation|vertical transportation safety|'
         r'assistive technology|safety technology|security device|chemical safety device|'
         r'electromechanical device|kitchen utensil|cleaning device|personal accessory|'
         r'personal grooming|packaging|office equipment|energy technology|renewable energy|'
         r'oil and gas|fuel'
         r')\b',
         'Engineering & Technology'),

        # Information & Computing
        (r'\b('
         r'computing|computer science|computer engineering|computer hardware|'
         r'computer networking|networking technology|software|internet technology|'
         r'information technology|mobile technology|wireless communication|'
         r'telecommunications|communication technology|communication device|'
         r'communication system|communications|blockchain technology|'
         r'artificial intelligence|quantum computing|cryptography|financial technology|'
         r'data storage|digital storage|optical storage|computer interface|'
         r'touchscreen technology'
         r')\b',
         'Information & Computing'),

        # Arts & Media
        (r'\b('
         r'music|musical instrument|film technology|photography|photographic process|'
         r'television technology|audio technology|audio device|audio recording device|'
         r'audio playback device|video games|recreational technology|toys?|sports equipment'
         r')\b',
         'Arts & Media'),

        # Social Sciences & Humanities
        (r'\b(anthropology|ethology|paleoanthropology)\b',
         'Social Sciences & Humanities'),

        # Mathematics
        (r'\b(mathematics|math)\b',
         'Mathematics'),
    ]

    for pattern, label in mapping_rules:
        if re.search(pattern, s):
            return label
    return 'Other'
    
def merge_dict_values(rows):
    """
    Merge a list of values (e.g., country, inventor, etc.) by taking unique values and joining them with semicolon.
    """
    merged = set()
    for item in rows:
        if isinstance(item, str):
            # Splitting possible composite entries
            parts = re.split(r'[;，,]', item)
            for part in parts:
                merged.add(part.strip())
        else:
            merged.add(str(item).strip())
    return "; ".join(sorted(merged))

# Define a set of stopwords to remove in text cleaning for invention names
stopwords = set(['the', 'for', 'of', 'and', 'in', 'on', 'origin', 'proposed', 'idea', 'first'])


### Read CSV files and merge data
This cell reads three separate CSV files from this directory and merges them into a single DataFrame for unified processing.


In [60]:
df1 = pd.read_csv('../../raw_data/clean_data/clean_data_yann_1.csv')
df2 = pd.read_csv('../../raw_data/clean_data/clean_data_yann_2.csv')
df3 = pd.read_csv('../../raw_data/clean_data/clean_data_yann_3.csv')
df4 = pd.read_csv('../../raw_data/clean_data/clean_data_balam.csv')

# Concatenate all data into one DataFrame
df = pd.concat([df1, df2, df3, df4], ignore_index=True)
print("number of origin data: ", len(df))

number of origin data:  823


Cleansing the data, removing punctuation and redundant spaces other than semicolons from each field


In [61]:
df_clean = df.copy()

# Punctuation cleaning for all string type fields
for col in df_clean.columns:
    df_clean[col] = df_clean[col].apply(remove_punctuation)

df_clean.head()


,Year,Country,Name of Invention,Name of Inventor,Category
0,1900,Germany,Quantum theory proposed,Planck,Physics
1,1901,AustrianAmerican,Discovery of human blood groups,Landsteiner,Medicine
2,1905,Germany,Waveparticle duality of light,Einstein,Physics
3,1905,Germany,Special theory of relativity,Einstein,Physics
4,1906,United Kingdom,Existence of vitamins proposed,Hopkins,Biochemistry


### Export all country names for inspection
Extract unique values by combining all country fields into a Series

In [62]:
all_countries = df_clean['Country'].dropna().unique()
print("List of original country names:")
for country in sorted(all_countries):
    print(country)


List of original country names:
Australia
Australia; Switzerland
Austria
Austria; Netherlands
Austria; Sweden
AustrianAmerican
Belgium
Belgium; France
Belgium; United States
Britain
Bulgaria
Canada
Canada; United States
China
Croatia
Czech
Czechia
Denmark
Egyptian; Korean
England
Finland
France
France; Germany; United Kingdom
France; United Kingdom
Germany
Germany; Austria
Germany; Canada
Germany; Denmark; Germany; Germany; Austria
Germany; France
Germany; Switzerland; United States
Germany; USA
Germany; United Kingdom
Germany; United States
Hungarian; British
Hungary
Hungary; United Kingdom
India
International
Ireland
Ireland; United States
Israel
Italy
Italy; Germany
Italy; United States
Japan
Japan; Dutch
Japan; USA
Mexico; USA
Multiple
Netherlands
Netherlands; Japan
Netherlands; Switzerland
New Zealand
New Zealand; United Kingdom
Norway
Norway; United States
Poland
Poland; Canada
Poland; France
Poland; Germany
Poland; United States
Romania
Russia
Russia; USA
Russia; United Kingdom


### Standardised Country Names
Define country similar name mapping (all keys are lowercase)

In [63]:
# Expanded mapping of variants and demonyms to a single canonical name
country_mapping = {
    "british": "United Kingdom",
    "britain": "United Kingdom",
    "england": "United Kingdom",
    "wales": "United Kingdom",
    "scotland": "United Kingdom",
    "uk": "United Kingdom",
    "united kingdom": "United Kingdom",
    "usa": "United States",
    "us": "United States",
    "american": "United States",
    "united states": "United States",
    "soviet union": "Russia",
    "austrian": "Austria",
    "hungarian": "Hungary",
    "egyptian": "Egypt",
    "korean": "South Korea",
    "dutch": "Netherlands",
    # special case: hyphenated demonyms
    "austrian-american": "Austria; United States",
    # preserve literal tokens
    "international": "International",
    "multiple": "Multiple"
}

# Apply the already-defined standardize_country function
df_clean['Country'] = df_clean['Country'].apply(lambda x: standardize_country(x, country_mapping))

# Print out the final sorted list of unique country names
std_countries = sorted(df_clean['Country'].dropna().unique())

print("Standardised list of country names:")
for country in std_countries:
    print(f"- {country}")

Standardised list of country names:
- Australia
- Australia; Switzerland
- Australia; United Kingdom
- Australia; United States
- Austria
- Austria; Denmark; Germany
- Austria; Germany
- Austria; Netherlands
- Austria; Sweden
- Austrianamerican
- Belgium
- Belgium; France
- Belgium; United Kingdom
- Belgium; United States
- Bulgaria
- Canada
- Canada; Germany
- Canada; Poland
- Canada; United Kingdom; United States
- Canada; United States
- China
- China; United States
- Croatia
- Czech
- Czechia
- Denmark
- Egypt; South Korea
- Finland
- France
- France; Germany
- France; Germany; United Kingdom
- France; Japan; United States
- France; Poland
- France; United Kingdom
- France; United States
- Germany
- Germany; Italy
- Germany; Poland
- Germany; Switzerland; United States
- Germany; United Kingdom
- Germany; United States
- Hungary
- Hungary; United Kingdom
- India
- International
- International; United States
- Ireland
- Ireland; United States
- Israel
- Israel; United Kingdom; Unit

### Export all original invention categories

In [64]:
all_categories = df_clean['Category'].dropna().unique()
print("List of original invention categories:")
for cat in sorted(all_categories):
    print(cat)

List of original invention categories:
3D Printing
Acoustics
Aeronautical Device
Aeronautical Vehicle
Aerospace
Aerospace Engineering
Agricultural Chemistry
Agricultural Engineering
Agricultural Fencing
Agricultural Implement
Agricultural Machinery
Agricultural Technology
Agriculture
Ammunition
Anthropology
Artificial Intelligence
Assistive Technology
Astronomical Theory
Astronomy
Astrophysics
Audio Device
Audio Playback Device
Audio Recording Device
Audio Technology
Automotive Technology
Aviation
Aviation Technology
Battery Technology
Biochemistry
Biology
Biotechnology
Blockchain Technology
Chaos Theory
Chemical Discovery
Chemical Element
Chemical Engineering
Chemical Process
Chemical Safety Device
Chemistry
Civil Engineering
Cleaning Device
Climate Science
Climatology
Clothing
Communication Device
Communication System
Communication Technology
Communications
Computer Engineering
Computer Hardware
Computer Interface
Computer Networking
Computer Science
Computing
Construction Engineerin

### Aggregate invention categories into general categories
Apply the generalize_category function to the Category column and create a new column General_Category

In [65]:
df_clean['General_Category'] = df_clean['Category'].apply(generalize_category)

# Print the statistics of the transformed categories
print("General Category Statistics:")
print(df_clean['General_Category'].value_counts())

General Category Statistics:
General_Category
Engineering & Technology              299
Physical Sciences                     128
Chemistry & Materials                  98
Information & Computing                92
Life Sciences & Medicine               87
Arts & Media                           46
Other                                  27
Agricultural Sciences & Technology     22
Earth & Environmental Sciences         18
Social Sciences & Humanities            5
Mathematics                             1
Name: count, dtype: int64


### Data preprocessing and normalization (unifying case, removing stopwords and whitespace, etc.)
Normalize 'Name of Invention' as a basis for later deduplication

In [66]:

df_clean['Invention_Norm'] = df_clean['Name of Invention'].apply(lambda x: normalize_text(x, stopwords=stopwords))
# Similarly, normalize 'Name of Inventor' if needed
df_clean['Inventor_Norm'] = df_clean['Name of Inventor'].apply(lambda x: normalize_text(x))
df_clean.head()


,Year,Country,Name of Invention,Name of Inventor,Category,General_Category,Invention_Norm,Inventor_Norm
0,1900,Germany,Quantum theory proposed,Planck,Physics,Physical Sciences,quantum theory,planck
1,1901,Austrianamerican,Discovery of human blood groups,Landsteiner,Medicine,Life Sciences & Medicine,discovery human blood groups,landsteiner
2,1905,Germany,Waveparticle duality of light,Einstein,Physics,Physical Sciences,waveparticle duality light,einstein
3,1905,Germany,Special theory of relativity,Einstein,Physics,Physical Sciences,special theory relativity,einstein
4,1906,United Kingdom,Existence of vitamins proposed,Hopkins,Biochemistry,Chemistry & Materials,existence vitamins,hopkins


### Perform data deduplication using a fuzzy matching algorithm

In [67]:
def deduplicate_df(df, threshold=90):
    """
    Use fuzzy matching to group similar invention names and merge rows.
    The merging is performed by taking the union of all other column values.
    """
    groups = []  # list to store groups, each group is a list of row indices
    # List of normalized invention names
    names = df['Invention_Norm'].tolist()
    used_idx = set()
    
    for idx, name in enumerate(names):
        if idx in used_idx:
            continue
        # Create a new group with the current index
        group = [idx]
        used_idx.add(idx)
        for jdx in range(idx+1, len(names)):
            if jdx in used_idx:
                continue
            # Compute fuzzy ratio between two normalized strings
            ratio = fuzz.ratio(name, names[jdx])
            if ratio >= threshold:
                group.append(jdx)
                used_idx.add(jdx)
        groups.append(group)
    
    # Merge groups: for each group, merge the corresponding rows
    merged_records = []
    for group in groups:
        # If the group has only one record, keep it as is
        if len(group) == 1:
            merged_records.append(df.iloc[group[0]])
        else:
            # Merge each column (join values with semicolons), and apply a strategy (e.g., min) for numeric fields like 'Year'
            merged = {}
            # For non-numeric fields, merge unique values
            for col in ['Year', 'Country', 'Name of Invention', 'Name of Inventor', 'Category', 'General_Category']:
                values = df.iloc[group][col].dropna().astype(str).tolist()
                merged[col] = merge_dict_values(values)
            # For normalized fields, keep the first item
            merged['Invention_Norm'] = df.iloc[group[0]]['Invention_Norm']
            merged_records.append(pd.Series(merged))
    
    return pd.DataFrame(merged_records)

# Apply the deduplication function
df_dedup = deduplicate_df(df_clean, threshold=90)
print("Number of records before deduplication:", len(df_clean))
print("Number of records after deduplication:", len(df_dedup))


Number of records before deduplication: 823
Number of records after deduplication: 757


In [68]:
# Keep only the first year if there are multiple (e.g. “1853; 1867”) and convert to int
df_dedup['Year'] = (
    df_dedup['Year']
    .astype(str)
    .str.split(';')   # split on semicolon
    .str[0]           # take the first piece
    .astype(int)      # convert to integer
)

# Now you can safely sort by Year
df_dedup.sort_values(by='Year', inplace=True)


### Output the final cleaned and deduplicated data, then save it

In [69]:
print("Preview of the cleaned and deduplicated data:")
display(df_dedup.head())

# Remove the temporary normalization columns
df_dedup.drop(['Invention_Norm', 'Inventor_Norm'], axis=1, inplace=True)

df_dedup.to_csv('../../raw_data/clean_data/clean_data_dd.csv', index=False)
print("The final cleaned data has been saved to 'clean_data_dd.csv'")


Preview of the cleaned and deduplicated data:


,Year,Country,Name of Invention,Name of Inventor,Category,General_Category,Invention_Norm,Inventor_Norm
Unnamed 18,1700,Italy,Piano; The First Piano,Bartolomeo Cristofori,Music,Arts & Media,piano,NaN
446,1701,United Kingdom,mechanical seed drill,Jethro Tull,Agricultural Machinery,Agricultural Sciences & Technology,mechanical seed drill,jethro tull
448,1709,United Kingdom,used coke to smelt iron,Abraham Darby,Metallurgical Engineering,Chemistry & Materials,used coke to smelt iron,abraham darby
Unnamed 19,1709,Germany; Poland,Alcohol Thermometer,Daniel Gabriel Fahrenheit; Gabriel Fahrenheit,Measurement Instrument; Physics,Physical Sciences,alcohol thermometer,NaN
141,1712,United Kingdom,The First Commercial Steam Engine,Thomas Newcomen,Mechanical Engineering,Engineering & Technology,commercial steam engine,thomas newcomen


The final cleaned data has been saved to 'clean_data_dd.csv'
